## Import packages

In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from scipy import spatial

## Load dataset

In [2]:
jobs = pd.read_csv('./dataset/dice_com-job_us_sample.csv')

## Building the model

In [18]:
def cosine_similarity(arr1,arr2):
    ans=1- spatial.distance.cosine(arr1,arr2)
    if(np.isnan(ans)):
        return 0
    else:
        return ans
class job_postings:    
    def __init__(self,link):
        self.df2=pd.read_csv(link)
        self.training_range=int(len(self.df2.loc[:,'uniq_id']))
    def match_profile(self,input_path,user_id,alpha,beta,flag=0):
        #Match a given user_id with all jobs in the database
        
        #Check if user id exists
        df=pd.read_csv(input_path+"domain_user_profile.csv",index_col='Respondent')
        #print(df.columns)
        matches=dict()
        if(flag==0):
            if(user_id in df.index):
                userdomain=df.loc[user_id,:]
                #print(userdomain)
                #If it does, retrieve the user profile from input_path
                df=pd.read_csv(input_path+"languages_profile_user.csv",index_col='Respondent')
                df.drop(['bash'], axis=1,inplace = True)
                userlanguages=df.loc[user_id,:]

                df=pd.read_csv(input_path+"frameworks_profile_user.csv",index_col='Respondent')
                userframeworks=df.loc[user_id,:]

                df=pd.read_csv(input_path+"platforms_profile_user.csv",index_col='Respondent')
                userplatforms=df.loc[user_id,:]

                df=pd.read_csv(input_path+"databases_profile_user.csv",index_col='Respondent')
                userdatabases=df.loc[user_id,:]

                userdomain=np.asarray(userdomain.fillna(0))
                userlanguages=np.asarray(userlanguages.fillna(0))
                userframeworks=np.asarray(userframeworks.fillna(0))
                userplatforms=np.asarray(userplatforms.fillna(0))
                userdatabases=np.asarray(userdatabases.fillna(0))
                #print(userdomain)
            else:
                print("error! user id not in Dataset")
            #If it doesn't,take user profile as input
        else:

            print("New user!Enter details..")
            name=input("Enter full name")
            skills=input("Enter skills(comma separated). These are programming languages, frameworks,platforms or databases you have experience with").split(",")
            domains=''
            flag=1
            while(1):
                print("Enter domain(s) of interest separated by commas(Names are case sensitive). Should be one of the following:")
                for i in df.columns:
                    print(i,end=",")
                domains=input().split(",")
                for domain in domains:
                    if(domain not in df.columns):
                        flag=0
                        break
                if(flag==1):
                    break
                else:
                    print("Please enter valid domain")
            #domains=list(map(lambda x:x.lower(),domains))
            skills=list(map(lambda x:x.lower(),skills))                

            userdomain=pd.DataFrame(columns=df.columns)
            dictionary=dict()
            for domain in domains:
                dictionary[domain]=1.0
            userdomain=userdomain.append(dictionary,ignore_index=True)


            df=pd.read_csv(input_path+"languages_profile_user.csv",index_col='Respondent')
            userlanguages=pd.DataFrame(columns=df.columns)
            dictionary=dict()
            for skill in skills:
                if(skill in df.columns):
                    dictionary[skill]=1.0
            userlanguages=userlanguages.append(dictionary,ignore_index=True)

            df=pd.read_csv(input_path+"frameworks_profile_user.csv",index_col='Respondent')
            userframeworks=pd.DataFrame(columns=df.columns)
            dictionary=dict()
            for skill in skills:
                if(skill in df.columns):
                    dictionary[skill]=1.0
            userframeworks=userframeworks.append(dictionary,ignore_index=True)

            df=pd.read_csv(input_path+"platforms_profile_user.csv",index_col='Respondent')
            userplatforms=pd.DataFrame(columns=df.columns)                
            dictionary=dict()
            for skill in skills:
                if(skill in df.columns):
                    dictionary[skill]=1.0
            userplatforms=userplatforms.append(dictionary,ignore_index=True)

            df=pd.read_csv(input_path+"databases_profile_user.csv",index_col='Respondent')
            userdatabases=pd.DataFrame(columns=df.columns)               
            dictionary=dict()
            for skill in skills:
                if(skill in df.columns):
                    dictionary[skill]=1.0
            userdatabases=userdatabases.append(dictionary,ignore_index=True)
            #print(userdomain)
            userdomain.head()
            userdomain=np.asarray(userdomain.iloc[0,:].fillna(0))
            userlanguages=np.asarray(userlanguages.iloc[0,:].fillna(0))
            userframeworks=np.asarray(userframeworks.iloc[0,:].fillna(0))
            userplatforms=np.asarray(userplatforms.iloc[0,:].fillna(0))
            userdatabases=np.asarray(userdatabases.iloc[0,:].fillna(0))
                
        jobdomain=pd.read_csv(input_path+"domain_job_profile.csv",index_col='uniq_id')
        joblanguages=pd.read_csv(input_path+'languages_profile_job.csv',index_col='uniq_id')
        jobframeworks=pd.read_csv(input_path+'frameworks_profile_job.csv',index_col='uniq_id')
        jobplatforms=pd.read_csv(input_path+'platforms_profile_job.csv',index_col='uniq_id')
        jobdatabases=pd.read_csv(input_path+'databases_profile_job.csv',index_col='uniq_id')
        
        #print(len(jobdomain.index),len(joblanguages.index))
        for i,j in zip(jobdomain.index,joblanguages.index):
            #print(i)
            domain=jobdomain.iloc[i,:].fillna(0)
            language=joblanguages.iloc[i,:].fillna(0)
            framework=jobframeworks.iloc[i,:].fillna(0)
            platform=jobplatforms.iloc[i,:].fillna(0)
            database=jobdatabases.iloc[i,:].fillna(0)
            #print('Uniq_id: ',joblanguages['uniq_id'])
            job_id=str(j)
            
            domain=np.asarray(domain)
            language=np.asarray(language)
            framework=np.asarray(framework)
            platform=np.asarray(platform)
            database=np.asarray(database)
            
            #print(language)
            #print(userlanguages)
            
            score=(alpha*cosine_similarity(domain,userdomain))+(beta*(cosine_similarity(language,userlanguages)+cosine_similarity(framework,userframeworks)+cosine_similarity(platform,userplatforms)+cosine_similarity(database,userdatabases)))
            matches[job_id]=score
            score=(alpha*cosine_similarity(domain,userdomain))+(beta*(cosine_similarity(language,userlanguages)+cosine_similarity(framework,userframeworks)+cosine_similarity(platform,userplatforms)+cosine_similarity(database,userdatabases)))
            
            #Initializing job profiles for later access
            self.job_domain=domain
            self.job_language=language
            self.job_framework=framework
            self.job_platform=platform
            self.job_database=database
            
            self.user_domain=userdomain
            self.user_language=userlanguages
            self.user_framework=userframeworks
            self.user_platform=userplatforms
            self.user_database=userdatabases
        matches=sorted(matches.items(),key=lambda x:x[1],reverse=True)
        
        recommendations=matches[:5]
        print("The top 5 Recommendations for User",user_id,"based on content-based filtering are:")
        
        # Recommendation dataframe
        ls = ['Respondent']
        for col in jobs.columns:
            ls.append(col)
        rows=pd.DataFrame(columns=ls)
        row=pd.DataFrame(columns=ls)
        count=0
        for i in recommendations:
            row.loc[count,'Respondent'] = user_id
            for col in jobs.columns:
                row[col] = jobs[jobs['uniq_id']==i[0]][col]
                #row[col] = self.df2[self.df2['uniq_id']==i[0]][col]
                
            #row=self.df2[self.df2['uniq_id']==i[0]]
            #rows[count]=np.asarray(row.values.T.tolist()[0])
            rows=rows.append(row.iloc[0])
            row=pd.DataFrame(columns=ls)
            count=count+1
            #print(row)
        #return rows
        #return recommendations
        for i in recommendations:
            print('Job Unique Id:',i[0])
        return rows,recommendations

In [19]:
obj=job_postings("./dataset/dice_com-job_us_sample.csv")

## Generating recommendations:

In [23]:
# For user = 4 & parameter for alpha=0.0 & beta=1.0
user_id = 3
rows,recommendations = obj.match_profile("./dataset/",user_id,alpha=0.0,beta=1.0)

The top 5 Recommendations for User 3 based on content-based filtering are:
Job Unique Id: 418ff92580b270ef4e7c14f0ddfc36b4
Job Unique Id: 8aec88cba08d53da65ab99cf20f6f9d9
Job Unique Id: 46baa1f69ac07779274bcd90b85d9a72
Job Unique Id: 3941b2f206ae0f900c4fba4ac0b18719
Job Unique Id: 45efa1f6bc65acc32bbbb953a1ed13b7


In [24]:
rows.head()

,Respondent,advertiserurl,company,employmenttype_jobstatus,jobdescription,jobid,joblocation_address,jobtitle,postdate,shift,site_name,skills,uniq_id
0,3,https://www.dice.com/jobs/detail/AUTOMATION-TE...,"Digital Intelligence Systems, LLC","C2H Corp-To-Corp, C2H Independent, C2H W2, 3 M...",Looking for Selenium engineers...must have sol...,Dice Id : 10110693,"Atlanta, GA",AUTOMATION TEST ENGINEER,1 hour ago,Telecommuting not available|Travel not required,NaN,SEE BELOW,418ff92580b270ef4e7c14f0ddfc36b4
1,3,https://www.dice.com/jobs/detail/Information-S...,University of Chicago/IT Services,Full Time,The University of Chicago has a rapidly growin...,Dice Id : 10114469,"Chicago, IL",Information Security Engineer,1 week ago,Telecommuting not available|Travel not required,NaN,"linux/unix, network monitoring, incident respo...",8aec88cba08d53da65ab99cf20f6f9d9
2,3,https://www.dice.com/jobs/detail/Business-Solu...,"Galaxy Systems, Inc.",Full Time,"GalaxE.SolutionsEvery day, our solutions affec...",Dice Id : CXGALXYS,"Schaumburg, IL",Business Solutions Architect,2 weeks ago,Telecommuting not available|Travel not required,NaN,"Enterprise Solutions Architecture, business in...",46baa1f69ac07779274bcd90b85d9a72
3,3,https://www.dice.com/jobs/detail/Java-Develope...,TransTech LLC,Full Time,Java DeveloperFull-time/direct-hireBolingbrook...,Dice Id : 10113627,"Bolingbrook, IL","Java Developer (mid level)- FT- GREAT culture,...",2 weeks ago,Telecommuting not available|Travel not required,NaN,Please see job description,3941b2f206ae0f900c4fba4ac0b18719
4,3,https://www.dice.com/jobs/detail/DevOps-Engine...,Matrix Resources,Full Time,Midtown based high tech firm has an immediate ...,Dice Id : matrixga,"Atlanta, GA",DevOps Engineer,48 minutes ago,Telecommuting not available|Travel not required,NaN,"Configuration Management, Developer, Linux, Ma...",45efa1f6bc65acc32bbbb953a1ed13b7


## Export to csv

In [6]:
#rows.to_csv("./dataset/recommendations.csv")

In [29]:
# For user = 5 & parameter for alpha=0.0 & beta=1.0
user_id = 3
rows,recommendations = obj.match_profile("./dataset/",user_id,alpha=0.7,beta=0.3)

The top 5 Recommendations for User 3 based on content-based filtering are:
Job Unique Id: 4a737efc3dd5fe84a532e94b609ee484
Job Unique Id: 12f3cf7b841e50a75d7fa547cedc27a6
Job Unique Id: 12b6b02d3498a1d0432d9669a50087ef
Job Unique Id: 4630a1e50e7e85a4364bfd6ccc65edda
Job Unique Id: e383459f6086249b588f5a4c6f5622a8


In [30]:
# For user = 5 & parameter for alpha=0.0 & beta=1.0
user_id = 7
rows,recommendations = obj.match_profile("./dataset/",user_id,alpha=0.001,beta=0.999)

The top 5 Recommendations for User 7 based on content-based filtering are:
Job Unique Id: 9c1dae8f8326ff44336cbc65c4145524
Job Unique Id: f7fac0c163a247d4f85c04e3dc823a7e
Job Unique Id: 3071e1d037c43c96e63d87b7f798904c
Job Unique Id: 3142c2dd6924df52d463d81ef93fb6e4
Job Unique Id: 0445fcb37ab17f686c025da15a98de52
